### Libraries

In [ ]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image

### fiji initialization

In [ ]:
scyjava.config.add_option('-Xmx60g')
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/sw/single_bact/202412/Fiji.app', mode='interactive')
ij.ui().showUI()
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

In [ ]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
rm = ij.RoiManager.getRoiManager()
ov = Overlay()

### Set working directory
Select the folder containing the different folders of your images

In [ ]:
# Dialog window to get the path of the working directory
root = tk.Tk()
root.withdraw()
# Set the path to the directory containing the images
directory_path = filedialog.askdirectory()
folder_list = os.listdir(directory_path)

### Flattening of images

In [ ]:
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
     # Find all the ".czi" files in the folder
    file_pattern = os.path.join(folder_path, "*.tif")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        #image = ij.io().open(file_path)
        #imp = ij.py.to_imageplus(image)
        imp = ij.IJ.openImage(file_path)
        # Change bit depth to 8-bit
        ij.IJ.run(imp, "8-bit", "")
        # Flatten the image
        flatten = imp.flatten()
        ij.IJ.run(flatten, "8-bit", "")
        results = ij.py.from_java(flatten)
        flatten_image = ij.py.to_dataset(results) 
        result_path = os.path.splitext(file_path)[0] + f"_flatten.tif"
        ij.io().save(flatten_image, result_path)
        print(f"Saving image {result_path}.")

### Cellpose initiation and ROI collection

In [ ]:
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "Image_CH2_flatten.tif")
    file_list = glob.glob(file_pattern)
    model = models.CellposeModel(gpu=True, model_type='Sty_fluo_high_1-28')
    #model = models.CellposeModel(gpu=True, model_type='sty_fluo_low_8')
        #model = models.CellposeModel(gpu=True, model_type='sty_BF_0406')
    for file_path in file_list:
        imgs = io.imread(file_path)
        channels = [[0,0]]
        #masks, flows, styles = model.eval(imgs, diameter=10, channels=channels)
        masks, flows, styles = model.eval(imgs, diameter=None, channels=channels)
        io.save_to_png(imgs, masks, flows, file_path)

In [ ]:
rm = ij.RoiManager.getRoiManager()
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    file_pattern = os.path.join(folder_path, "Image_CH2_flatten.tif")
    file_list = glob.glob(file_pattern)
    for file_path in file_list: 
        imp = ij.IJ.openImage(file_path)
        basename = os.path.basename(file_path)
        f_name = os.path.splitext(basename)[0]
        input_txt = Path(f"{folder_path}/{f_name}_cp_outlines.txt")
        txt_fh = open(input_txt, 'r')
        for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Save", f"{folder_path}/" + f"RoiSet.zip")
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Delete")

### Measurement of Fluorescence on single bacteria
This will go across every image and every channel you have available

In [ ]:
rm = ij.RoiManager.getRoiManager()
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    yellow = f"{folder_path}/Image_CH2_flatten.tif"
    red = f"{folder_path}/Image_CH3_flatten.tif"
    blue = f"{folder_path}/Image_CH4_flatten.tif"
    if os.path.exists(yellow):
        imp = ij.io().open(yellow)
        ij.ui().show(imp)
        ij.IJ.run("Set Scale...", "distance=1 known=0.12581 unit=um")
        set_string = f'Set Measurements...'
        measure_string = f'area mean integrated stack redirect=None decimal=2'
        ij.IJ.run(set_string, measure_string)
        input_ROI = Path(f"{folder_path}/RoiSet.zip")
        rm.open(f"{input_ROI}")
        measure = """
        nbArea=roiManager("count")
        for (i=0; i<nbArea; i++) {
    		roiManager("Select", i);
            run("Measure");	
    	}
        roiManager('Select All');
        roiManager("Delete");
        close("*");  
        """
        ij.py.run_macro(measure)
        results = os.path.basename(folder_path)
        results = os.path.splitext(results)[0]
        output_path = f"{folder_path}/{results}_yellow.csv"
        saving = ij.IJ.saveAs("Results", output_path)
        ij.IJ.run("Clear Results")
        
    if os.path.exists(red):
        imp = ij.io().open(red)
        ij.ui().show(imp)
        ij.IJ.run("Set Scale...", "distance=1 known=0.12581 unit=um")
        set_string = f'Set Measurements...'
        measure_string = f'area mean integrated stack redirect=None decimal=2'
        ij.IJ.run(set_string, measure_string)
        input_ROI = Path(f"{folder_path}/RoiSet.zip")
        rm.open(f"{input_ROI}")
        measure = """
        nbArea=roiManager("count")
        for (i=0; i<nbArea; i++) {
    		roiManager("Select", i);
            run("Measure");	
    	}
        roiManager('Select All');
        roiManager("Delete");
        close("*");  
        """
        ij.py.run_macro(measure)
        results = os.path.basename(folder_path)
        results = os.path.splitext(results)[0]
        output_path = f"{folder_path}/{results}_red.csv"
        saving = ij.IJ.saveAs("Results", output_path)
        ij.IJ.run("Clear Results")
        
    if os.path.exists(blue):
        imp = ij.io().open(blue)
        ij.ui().show(imp)
        ij.IJ.run("Set Scale...", "distance=1 known=0.12581 unit=um")
        set_string = f'Set Measurements...'
        measure_string = f'area mean integrated stack redirect=None decimal=2'
        ij.IJ.run(set_string, measure_string)
        input_ROI = Path(f"{folder_path}/RoiSet.zip")
        rm.open(f"{input_ROI}")
        measure = """
        nbArea=roiManager("count")
        for (i=0; i<nbArea; i++) {
    		roiManager("Select", i);
            run("Measure");	
    	}
        roiManager('Select All');
        roiManager("Delete");
        close("*");  
        """
        ij.py.run_macro(measure)
        results = os.path.basename(folder_path)
        results = os.path.splitext(results)[0]
        output_path = f"{folder_path}/{results}_blue.csv"
        saving = ij.IJ.saveAs("Results", output_path)
        ij.IJ.run("Clear Results")

### Concatenation of the different csv files into one called "grouped" per folder

In [ ]:
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    results = os.path.basename(folder_path)
    results = os.path.splitext(results)[0]
    input_csv_yellow = Path(f"{folder_path}/{results}_yellow.csv")
    input_csv_cyan = Path(f"{folder_path}/{results}_blue.csv")
    input_csv_red = Path(f"{folder_path}/{results}_red.csv")
    if os.path.exists(input_csv_red):
        #first marker csv
        df1 = pandas.read_csv(input_csv_yellow)
        df1.rename(columns={'Mean': 'Mean_mGreenLantern'}, inplace=True)
        df1 = df1.drop(columns = [df1.columns[0], df1.columns[3]], axis =1)
        
        #second marker csv
        df2 = pandas.read_csv(input_csv_red)
        df2.rename(columns={'Mean': 'Mean_red'}, inplace=True)
        df2 = df2.drop(columns = [df2.columns[0],df2.columns[1], df2.columns[3]], axis =1)
        
        #create new merged dataframe:
        final_results = pandas.concat([df1, df2], axis = 1)
        #final_results = final_results.iloc[:, [1, 0, 2]]
        
        # add a column for track_ids (based on grouped.csv)
        final_path = Path(f"{folder_path}/{results}_grouped.csv")
        final_results.to_csv(final_path)

    elif os.path.exists(input_csv_cyan):
        #first marker csv
        df1 = pandas.read_csv(input_csv_yellow)
        df1.rename(columns={'Mean': 'Mean_mGreenLantern'}, inplace=True)
        df1 = df1.drop(columns = [df1.columns[0], df1.columns[3]], axis =1)
        
        #second marker csv
        df2 = pandas.read_csv(input_csv_cyan)
        df2.rename(columns={'Mean': 'Mean_CFP'}, inplace=True)
        df2 = df2.drop(columns = [df2.columns[0],df2.columns[1], df2.columns[3]], axis =1)
        
        #create new merged dataframe:
        final_results = pandas.concat([df1, df2], axis = 1)
        #final_results = final_results.iloc[:, [1, 0, 2]]
        
        # add a column for track_ids (based on grouped.csv)
        final_path = Path(f"{folder_path}/{results}_grouped.csv")
        final_results.to_csv(final_path)

In [ ]:
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    results = os.path.basename(folder_path)
    results = os.path.splitext(results)[0]
    input_csv_yellow = Path(f"{folder_path}/{results}_yellow.csv")
    input_csv_cyan = Path(f"{folder_path}/{results}_blue.csv")
    input_csv_red = Path(f"{folder_path}/{results}_red.csv")
    if os.path.exists(input_csv_red):
        #first marker csv
        df1 = pandas.read_csv(input_csv_yellow)
        df1.rename(columns={'Mean': 'Mean_mGreenLantern'}, inplace=True)
        df1 = df1.drop(columns = [df1.columns[0], df1.columns[3]], axis =1)
        
        #second marker csv
        df2 = pandas.read_csv(input_csv_cyan)
        df2.rename(columns={'Mean': 'Mean_CFP'}, inplace=True)
        df2 = df2.drop(columns = [df2.columns[0],df2.columns[1], df2.columns[3]], axis =1)

        # third marker csv
        df3 = pandas.read_csv(input_csv_red)
        df3.rename(columns={'Mean': 'Mean_mCherry'}, inplace=True)
        df3 = df3.drop(columns = [df3.columns[0], df3.columns[1], df3.columns[3]], axis =1)
        
        #create new merged dataframe:
        final_results = pandas.concat([df1, df2, df3], axis = 1)
        #final_results = final_results.iloc[:, [1, 0, 2]]
        
        # add a column for track_ids (based on grouped.csv)
        final_path = Path(f"{folder_path}/{results}_grouped.csv")
        final_results.to_csv(final_path)

    elif os.path.exists(input_csv_cyan):
        #first marker csv
        df1 = pandas.read_csv(input_csv_yellow)
        df1.rename(columns={'Mean': 'Mean_mGreenLantern'}, inplace=True)
        df1 = df1.drop(columns = [df1.columns[0], df1.columns[3]], axis =1)
        
        #second marker csv
        df2 = pandas.read_csv(input_csv_cyan)
        df2.rename(columns={'Mean': 'Mean_CFP'}, inplace=True)
        df2 = df2.drop(columns = [df2.columns[0],df2.columns[1], df2.columns[3]], axis =1)
        
        #create new merged dataframe:
        final_results = pandas.concat([df1, df2], axis = 1)
        #final_results = final_results.iloc[:, [1, 0, 2]]
        
        # add a column for track_ids (based on grouped.csv)
        final_path = Path(f"{folder_path}/{results}_grouped.csv")
        final_results.to_csv(final_path)

In [ ]:
ij.IJ.run("Translate...", "x=4 y=-9 interpolation=None")

### foci quantification
If the segmentation worked...

In [ ]:
threshold_low = 40
threshold_high = 255
w = scyjava.jimport('ij.WindowManager')
ij.IJ.run("Set Measurements...", f'area limit redirect=None decimal=2')
for folder in os.listdir(directory_path):
    image_path = f"{directory_path}/{folder}/Image_CH3_flatten.tif"
    imp = ij.IJ.openImage(image_path)
    imp.show()
    ij.IJ.setAutoThreshold(imp, "Default dark")
    ij.IJ.setRawThreshold(imp, threshold_low, threshold_high)
    ij.IJ.run(imp, "Convert to Mask", "background=Dark black");
    input_ROI = Path(f"{directory_path}/{folder}/RoiSet.zip")
    rm.open(f"{input_ROI}")
    roi_count = rm.getCount()
    for i in range(0, roi_count-1):
        rm.select(i)
        ij.IJ.run(imp, "Analyze Particles...", f"circularity=0.80-1.00 summarize");
    window_name = f"Summary"
    ij.IJ.selectWindow(window_name)
    ij.IJ.renameResults(f"{window_name}", "Results")
    output_path = Path(f"{directory_path}/{folder}/{folder}_puncta_summary.csv").as_posix()
    ij.IJ.saveAs("Results", output_path)
    ij.IJ.run("Clear Results")
    w.getWindow("Results").close()
    rm.runCommand("Reset")
    ij.py.run_macro('close("*");')

### Single bacterium tracking

In [ ]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
ov = Overlay()
Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
TmXmlReader = scyjava.jimport('fiji.plugin.trackmate.io.TmXmlReader')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')
IJRoiExporter = scyjava.jimport('fiji.plugin.trackmate.action.IJRoiExporter')
rm = ij.RoiManager.getRoiManager()

In [ ]:
def track_cells(channels_image, directory_path, dsettings, tsettings, distance=1, known=0.12581, unit="µm"):
    '''
    This function leverages ImageJ's Trackmate plugin to track the cells in channels.tif. It uses a custom-trained Cellpose model for segmentation (with detection parameters specified in dsettings), and the LAP Tracker algorithm for tracking (with tracking parameters specified in tsettings).
    The function saves the Trackmate XML model as well as a CSV file of the tracks as output. The function also takes as input the scale of the image.
    '''
    # Open Image
    imp = ij.IJ.openImage(channels_image)
    imp.show()
    ij.py.run_macro(f'run("Set Scale...", "distance={distance} known={known} unit={unit}");')    
    # Create Model
    model = Model()
    settings = Settings(imp)   
    # Detector
    settings.detectorFactory = CellposeDetectorFactory()
    for parameter, value in dsettings.items():
        settings.detectorSettings[parameter] = value

    filter1 = FeatureFilter('QUALITY', quality_filter, True)
    settings.addSpotFilter(filter1)
    # Tracker
    settings.trackerFactory = SparseLAPTrackerFactory()
    settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
    for parameter, value in tsettings.items():
        settings.trackerSettings[parameter] = value
    # Execute Tracking
    trackmate = TrackMate(model, settings)
    ok = trackmate.checkInput()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    ok = trackmate.process()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    selectionModel = SelectionModel(model)  
    # Display
    ds = DisplaySettingsIO.readUserDefault()
    displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
    displayer.render()
    displayer.refresh()
    trackscheme = TrackScheme(model, selectionModel, ds)
    trackscheme.render()
    # Save Data
    outFile = Path(directory_path+"exportModel.xml")
    writer = TmXmlWriter(outFile)
    writer.appendModel(model)
    writer.appendSettings(settings)
    writer.writeToFile()
    csvFileTracks = Path(directory_path+"exportTracks.csv")
    trackTableView = TrackTableView(model, selectionModel, ds, channels_image)
    trackTableView.getSpotTable().exportToCsv(csvFileTracks)
    return csvFileTracks, model, imp

def sort_trackmate_export(tm_tracks_csv, directory_path):
    '''
    This function sorts the exported CSV file from Trackmate so that the cells are ordered by track ID and frame number. It saves the sorted file to the directory path.
    '''
    df = pandas.read_csv(tm_tracks_csv)
    df.drop([0, 1, 2], axis=0, inplace=True)
    df = df.reset_index(drop=True)
    for index, row in df.iterrows():
        df.at[index, 'TRACK_ID'] = int(row['TRACK_ID'])
        df.at[index, 'FRAME'] = int(row['FRAME'])
    df.sort_values(by=['TRACK_ID', 'FRAME'], inplace=True, ignore_index=True)
    df.to_csv(directory_path+"/exportTracks_edited.csv", index=False)
    output = directory_path+"/exportTracks_edited.csv"
    return output

def get_cell_rois(directory_path, model, imp):
    '''
    This function exports the cell ROIs from Trackmate and saves them in a zip file. 
    '''
    model.setLogger(Logger.IJ_LOGGER)
    logger = Logger.IJ_LOGGER
    exporter = IJRoiExporter(imp, logger)
    exporter.export(model.getSpots().iterable(True))
    rm.save(directory_path+"/RoiSet_cells.zip");
    rm.runCommand(imp,"Delete");
    with ZipFile(directory_path+"/RoiSet_cells.zip", 'r') as zip: 
        zip.extractall(path=directory_path+"/cell_rois")
    output = directory_path+"/cell_rois/"
    return output

def create_tracks_csv(sorted_tm_tracks_csv, directory_path):
    '''
    This function creates a CSV file (da_Tracks.csv) where each row is a separate track and lists the ROI Manager indices that correspond to cell ROIs in that track.
    '''
    data = pandas.read_csv(sorted_tm_tracks_csv)
    file = open(directory_path+'/da_Tracks.csv', 'w', newline='')
    csv_writer = csv.writer(file)
    current_track = 0
    first = True
    string = ''
    for index, row in data.iterrows():
        if int(row['TRACK_ID']) == current_track:
            if first:
                string = f'{index+1}'
                first = False
            else:
                string = string + ", " + f'{index+1}'
        else:
            csv_writer.writerow([string])
            string = f'{index+1}'
            current_track += 1  
    file.close()
    output = directory_path+"/da_Tracks.csv"
    return output

def get_total_fluorescence(directory_path, sum_channels, imp, cells_desired, channel_dict, target_channel):
    # target channel should be a string ex. LC3
    imp = ij.IJ.openImage(sum_channels)
    imp.show()
    imp.setC(channel_dict[target_channel])
    set_string = f'Set Measurements...'
    measure_string = f'mean integrated stack redirect=None decimal=2'
    ij.IJ.run(set_string, measure_string)
    tracks_csv = directory_path+'da_Tracks.csv'
    df = pandas.read_csv(tracks_csv, header=None)
    for cell_desired in cells_desired:
        nums = []
        if isinstance(cell_desired, int):
            row = df.iloc[cell_desired-1]
            nums = row[0].split(", ")
        elif isinstance(cell_desired, list):
            for c in cell_desired:
                row = df.iloc[c-1]
                nums += row[0].split(", ")
        arr = []
        for x in nums:
            arr.append(int(x))
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            ij.IJ.run('Measure')    
        output = Path(f"{directory_path}/misc/cell{cell_desired}_{target_channel}_fluo.csv").as_posix()
        saving = ij.IJ.saveAs("Results", output)
        ij.IJ.run("Clear Results")
    ij.py.run_macro('close("*");')
    return output

def add_rois(directory_path, sorted_bact_tracks_csv, roiset):
    '''
    This function adds cell ROIs into the ImageJ ROI Manager *in order*. If the sorted ROI zip file already exists, then it directly loads it into the ROI Manager. Otherwise, it references sorted_tracks_csv and saves the sorted zip file at the end.
    It is imperative to run this function to load the ROI Manager (as opposed to dragging and dropping the RoiSet into ImageJ GUI) in order to preserve the order of the ROIS and ensure consistency with ROI labels given in da_Tracks.csv.
    '''
    rm.runCommand("Reset")
    if os.path.exists(directory_path+"/sorted_RoiSet_bact.zip"):
        rm.open(directory_path+"/sorted_RoiSet_bact.zip")
    else:
        df = pandas.read_csv(sorted_bact_tracks_csv)
        for index, row in df.iterrows():
            file = row['LABEL'] + '.roi'
            rm.open(roiset+file)
        rm.save(directory_path+"/sorted_RoiSet_bact.zip")

def create_tracks_csv(sorted_tm_tracks_csv, directory_path):
    '''
    This function creates a CSV file (da_Tracks.csv) where each row is a separate track and lists the ROI Manager indices that correspond to cell ROIs in that track.
    '''
    data = pandas.read_csv(sorted_tm_tracks_csv)
    file = open(directory_path+'/da_Tracks.csv', 'w', newline='')
    csv_writer = csv.writer(file)
    current_track = 0
    first = True
    string = ''
    for index, row in data.iterrows():
        if int(row['TRACK_ID']) == current_track:
            if first:
                string = f'{index+1}'
                first = False
            else:
                string = string + ", " + f'{index+1}'
        else:
            csv_writer.writerow([string])
            string = f'{index+1}'
            current_track += 1  
    file.close()
    output = directory_path+"da_Tracks.csv"
    return output

def add_rois(directory_path, sorted_bact_tracks_csv, roiset):
    '''
    This function adds cell ROIs into the ImageJ ROI Manager *in order*. If the sorted ROI zip file already exists, then it directly loads it into the ROI Manager. Otherwise, it references sorted_tracks_csv and saves the sorted zip file at the end.
    It is imperative to run this function to load the ROI Manager (as opposed to dragging and dropping the RoiSet into ImageJ GUI) in order to preserve the order of the ROIS and ensure consistency with ROI labels given in da_Tracks.csv.
    '''
    rm.runCommand("Reset")
    if os.path.exists(directory_path+"/sorted_RoiSet_bact.zip"):
        rm.open(directory_path+"/sorted_RoiSet_bact.zip")
    else:
        df = pandas.read_csv(sorted_bact_tracks_csv)
        for index, row in df.iterrows():
            file = row['LABEL'] + '.roi'
            rm.open(roiset+file)
        rm.save(directory_path+"/sorted_RoiSet_bact.zip")

def get_cell_features(directory_path, distance, known, unit, cells_desired, features=f'area centroid stack'):
    '''
    This function obtains measurements for cells listed in cells_desired, including the area and centroid of each cell at each time step. 
    The measurements to be obtained are specified in the variable features, which should be a string with no spaces; e.g. 'area centroid stack'
    The function saves a separate CSV file with the measurements for each cell in the misc directory.
    '''
    image_path = Path(f"{directory_path}/channels.tif").as_posix()
    imp = ij.IJ.openImage(image_path)
    ij.ui().show(imp)
    rm.runCommand("Reset")
    add_rois(directory_path, sorted_tm_tracks_csv, roiset)
    ij.py.run_macro(f'run("Set Scale...", "distance={distance} known={known} unit={unit}");')    
    set_string = f'Set Measurements...'
    measure_string = features + f' redirect=None decimal=2'
    ij.IJ.run(set_string, measure_string)
    w = scyjava.jimport('ij.WindowManager')
    df = pandas.read_csv(tracks_csv, header=None)
    for cell_desired in cells_desired:
        nums = []
        if isinstance(cell_desired, int):
            row = df.iloc[cell_desired-1]
            nums = row[0].split(", ")
        elif isinstance(cell_desired, list):
            for c in cell_desired:
                row = df.iloc[c-1]
                nums += row[0].split(", ")
        arr = []
        for x in nums:
            arr.append(int(x))
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            ij.IJ.run('Measure')
        misc_path = directory_path+'/misc/'
        if not os.path.exists(misc_path):
            os.makedirs(misc_path)        
        output = Path(f"{misc_path}bact{cell_desired}_numbers.csv").as_posix()
        saving = ij.IJ.saveAs("Results", output)
        ij.IJ.run("Clear Results")
    ij.py.run_macro('close("*");')
    ij.IJ.selectWindow("Results")
    ij.IJ.run("Close")
    return misc_path

In [ ]:
dsettings = { # specify parameters for the detection step in Trackmate (Threshold Detector)
    'TARGET_CHANNEL' : ij.py.to_java(0),
    'SIMPLIFY_CONTOURS' : True,
    'CELLPOSE_MODEL' : PretrainedModel.CUSTOM,
    'CELLPOSE_MODEL_FILEPATH' : "/Users/MrWor/.cellpose/models/Sty_fluo_high_1-28",
    'USE_GPU' : True,
    'OPTIONAL_CHANNEL_2' : ij.py.to_java(0),
    'CELL_DIAMETER' : 2.0,
    'CELLPOSE_PYTHON_FILEPATH' : "/Users/MrWor/anaconda3/envs/cellpose/python.exe",
}
quality_filter = 45
tsettings = { # specify parameters for the tracking step in Trackmate (LAP Tracker)
    'LINKING_MAX_DISTANCE' : 2.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 2.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
    'MERGING_MAX_DISTANCE' : 15.0,
}

# These are the parameters to fill when running the "set scale..." in fiji. This is crucial to make sure the distances during the tracking stay coherent along the workflow
distance = 1 
known = 0.12581
unit = "µm"

In [ ]:
channels_image = directory_path+'channels.tif'
max_channels = directory_path+'max_channels.tif'
tracks_csv = directory_path+'/da_Tracks.csv'
sorted_tm_tracks_csv = directory_path+'/exportTracks_edited.csv'
tm_tracks_csv = directory_path+'/exportTracks.csv'
roiset = directory_path+'/cell_rois/'
bact = directory_path+'bact/'
other = directory_path+'other_signals/'
grouped = directory_path+'bact/grouped/'
measurements = directory_path+'signal_measurement/'
misc_path = directory_path+'misc/'

In [ ]:
channels_image = f"{directory_path}/Merged-2.tif"

In [ ]:
# run the segmentation and tracking:
tm_tracks_csv, model, imp = track_cells(channels_image, directory_path, dsettings, tsettings, distance, known, unit)
# This function extract the ROIs from the trackmate results, and saves them in a directory named 'cell_rois'
roiset = get_cell_rois(directory_path, model, imp)

In [ ]:
# this sorts the tracks in the 'exportTracks.csv' file
sorted_tm_tracks_csv = sort_trackmate_export(tm_tracks_csv, directory_path)

In [ ]:
# for each tracks, this extract the index+1 of each spot to put 
tracks_csv = create_tracks_csv(sorted_tm_tracks_csv, directory_path)

In [ ]:
# This function adds the sorted ROIs of each cell in each tracks in the ROI manager
add_rois(directory_path, sorted_tm_tracks_csv, roiset)

In [ ]:
roiset = get_cell_rois(directory_path)

In [ ]:
conposite_path = f"{directory_path}/Merged.tif"
numbers_path = f"{directory_path}/results/"
if not os.path.exists(numbers_path):
    os.makedirs(numbers_path)
imp = ij.IJ.openImage(conposite_path)
imp.show()

In [ ]:
imp.setC(3)
ij.IJ.run("Set Scale...", "distance=1 known=0.12581 unit=um")
set_string = f'Set Measurements...'
measure_string = f'mean area integrated stack redirect=None decimal=2'
ij.IJ.run(set_string, measure_string)
cells_undesired = []
df = pandas.read_csv(tracks_csv, header=None)
for n in range(len(df)):
    nums = []
    row = df.iloc[n]
    nums = row[0].split(", ")
    arr = []
    for x in nums:
        arr.append(int(x))
    if len(arr) == 16:
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            ij.IJ.run('Measure')    
        output = Path(f"{numbers_path}bact{n}_numbers_mGL.csv").as_posix()
        saving = ij.IJ.saveAs("Results", output)
        ij.IJ.run("Clear Results")
#ij.py.run_macro('close("*");')

In [ ]:
#tracks_csv = directory_path+'/da_Tracks.csv'
df = pandas.read_csv(tracks_csv)
print(len(df))

In [ ]:
row = df.iloc[9]
nums = row[0].split(", ")
arr = []
for x in nums:
    arr.append(int(x))
print(arr)

In [ ]:
imp.setC(2)
set_string = f'Set Measurements...'
measure_string = f'mean area integrated stack redirect=None decimal=2'
ij.IJ.run(set_string, measure_string)
cells_undesired = [8, 91, 205, 208, 119, 240, 233, 275, 254, 274, 273, 270, 280,
                  286, 2, 5, 12, 14, 227, 228, 237, 160, 58, 75, 199, 210, 243]
df = pandas.read_csv(tracks_csv, header=None)
for n in range(len(df)):
    if n in cells_undesired:
        continue
    else:
        nums = []
        row = df.iloc[n]
        nums = row[0].split(", ")
        arr = []
        for x in nums:
            arr.append(int(x))
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            ij.IJ.run('Measure')    
        output = Path(f"{numbers_path}bact{n}_numbers_mCFP.csv").as_posix()
        saving = ij.IJ.saveAs("Results", output)
        ij.IJ.run("Clear Results")
#ij.py.run_macro('close("*");')

In [ ]:
imp.setC(3)
set_string = f'Set Measurements...'
measure_string = f'mean area integrated stack redirect=None decimal=2'
ij.IJ.run(set_string, measure_string)
cells_undesired = [8, 91, 205, 208, 119, 240, 233, 275, 254, 274, 273, 270, 280,
                  286, 2, 5, 12, 14, 227, 228, 237, 160, 58, 75, 199, 210, 243]
df = pandas.read_csv(tracks_csv, header=None)
for n in range(len(df)):
    if n in cells_undesired:
        continue
    else:
        nums = []
        row = df.iloc[n]
        nums = row[0].split(", ")
        arr = []
        for x in nums:
            arr.append(int(x))
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            ij.IJ.run('Measure')    
        output = Path(f"{numbers_path}bact{n}_numbers_mGL.csv").as_posix()
        saving = ij.IJ.saveAs("Results", output)
        ij.IJ.run("Clear Results")
#ij.py.run_macro('close("*");')

In [ ]:
conposite_path = f"{directory_path}/composite.tif"
numbers_path = f"{directory_path}/results/"
dossier = os.path.basename(directory_path)
area = []
frames = []
rawintden = []
file_pattern2 = os.path.join(numbers_path, "*.csv")
file_list2 = glob.glob(file_pattern2)
for csv in file_list2:
    df = pandas.read_csv(csv)
    norm_raw = []
    basename = os.path.basename(csv)
    corename = os.path.splitext(basename)[0]
    for i in range(len(df)):
        ratio = df['RawIntDen'][i] / df['Area'][i]
        norm_raw.append(ratio)
    df[f"{corename}_R/A"] = norm_raw
    rawintden.append(df[f"{corename}_R/A"])
    if basename.endswith("mGL.csv"):
        df.rename(columns={'Area': f"{corename}_area"}, inplace=True)
        df.rename(columns={'Frame': f"{corename}_Frames"}, inplace=True)
        rawintden.append(df[f"{corename}_area"])
        rawintden.append(df[f"{corename}_Frames"])
result_rawintden = pandas.concat(rawintden, axis=1)
#results_area = pandas.concat(area, axis=1)
#results_frames = pandas.concat(frames, axis=1)
result_rawintden.to_csv(f"{directory_path}/normRaw_{dossier}.csv")
#results_area.to_csv(f"{directory_path}/area_{dossier}.csv")
#results_frames.to_csv(f"{directory_path}/frames_{dossier}.csv")

In [ ]:
conposite_path = f"{directory_path}/composite.tif"
numbers_path = f"{directory_path}/results/"
dossier = os.path.basename(directory_path)
area = []
frames = []
rawintden = []
file_pattern2 = os.path.join(numbers_path, "*CFP.csv")
file_list2 = glob.glob(file_pattern2)
for csv in file_list2:
    df = pandas.read_csv(csv)
    norm_raw = []
    basename = os.path.basename(csv)
    corename = os.path.splitext(basename)[0]
    for i in range(len(df)):
        ratio = df['RawIntDen'][i] / df['Area'][i]
        norm_raw.append(ratio)
    df[f"{corename}_R/A"] = norm_raw
    rawintden.append(df[f"{corename}_R/A"])
result_rawintden = pandas.concat(rawintden, axis=1)
df_transposed = result_rawintden.transpose()
df_transposed.rename(columns={df_transposed.columns[0]: 't0', 
                           df_transposed.columns[1]: 't1',  
                           df_transposed.columns[2]: 't2', 
                           df_transposed.columns[3]: 't3', 
                           df_transposed.columns[4]: 't4', 
                           df_transposed.columns[5]: 't5',
                           df_transposed.columns[6]: 't6',
                           df_transposed.columns[7]: 't7',
                           df_transposed.columns[8]: 't8',
                           df_transposed.columns[9]: 't9', 
                           df_transposed.columns[10]: 't10', 
                           df_transposed.columns[11]: 't11', 
                           df_transposed.columns[12]: 't12', 
                           df_transposed.columns[13]: 't13'}, inplace=True)
df_transposed.to_csv(f"{directory_path}/normRaw_CFP.csv")
